In [ ]:
!pip install gym_super_mario_bros==7.3.0 nes_py

In [ ]:
!pip install stable-baselines3[extra]

In [ ]:
conda update --all

In [ ]:
conda install pytorch torchvision torchaudio -c pytorch

In [ ]:
conda install pytorch torchvision torchaudio cudatoolkit=11.3 -c pytorch

In [ ]:
conda install pytorch torchvision torchaudio cudatoolkit=10.2 -c pytorch-lts

In [ ]:
conda install freetype=2.10.4

In [1]:
import gym

# Import the game
import gym_super_mario_bros

# Import the joypad wrapper
from nes_py.wrappers import JoypadSpace

# Import the simplified controls
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT, RIGHT_ONLY 

In [2]:
# Import frame stack and grayscaling wrapper
from gym.wrappers import FrameStack, GrayScaleObservation

# Import vectorization wrappers
from stable_baselines3.common.vec_env import VecFrameStack, DummyVecEnv

# Import Matplotlib to show the impact of stack framing
from matplotlib import pyplot as plt

In [ ]:
# We will use a subset of possible actions
SIMPLE_MOVEMENT
#RIGHT_ONLY 
#CUSTUM_RIGHT_ONLY = [['right', 'B'], ['right', 'A', 'B']]
#CUSTUM_RIGHT_ONLY

In [ ]:
class CustomReward(gym.Wrapper):
    def __init__(self, env):
        super(CustomReward, self).__init__(env)
        self._current_score = 0

    def step(self, action):
        state, reward, done, info = self.env.step(action)
        reward += (info["score"] - self._current_score) / 40.0
        self._current_score = info["score"]
        if done:
            if info["flag_get"]:
                reward += 350.0
            else:
                reward -= 50.0
        return state, reward / 10.0, done, info

In [ ]:
class CustomRewardNoMovingRightReward(gym.Wrapper):
    def __init__(self, env):
        super(CustomRewardNoMovingRightReward, self).__init__(env)
        self._current_score = 0

        # starting point
        self._current_x_pos = 40

        self._max_x_pos_memory = 0
        self._previous_x_pos_memory = 0
        self._steps_run_wrong_direction = 0

    def step(self, action):
        state, reward, done, info = self.env.step(action)
        score_diff = info["score"] - self._current_score
        reward += score_diff / 10.0

        if self._max_x_pos_memory < info["x_pos"]:
            self._max_x_pos_memory = info["x_pos"]
        else:
            self._steps_run_wrong_direction += 1

        #  Handle when the agent hits the left wall
        standstill = self._previous_x_pos_memory == info["x_pos"]
        if standstill:
            self._steps_run_wrong_direction += 1

        self._previous_x_pos_memory = info["x_pos"]

        # reward movement also for left direction /normal reward for left is -3
        # Make sure to force progress after moving in the wrong direction too long.
        if info["x_pos"] < self._current_x_pos and self._steps_run_wrong_direction < 750:
            reward += 4
        elif standstill and self._steps_run_wrong_direction > 750:
            reward -= 2
        elif score_diff > 1:
            self._steps_run_wrong_direction = 0

        self._current_score = info["score"]
        self._current_x_pos = info["x_pos"]

        if done:
            if info["flag_get"]:
                reward += 350.0
            else:
                reward -= 100.0
                self._current_x_pos = 40
        return state, reward, done, info

In [ ]:
class CustomRewardNoMovingRightReward(gym.Wrapper):
    def __init__(self, env):
        super(CustomRewardNoMovingRightReward, self).__init__(env)
        self._current_score = 0
        self._current_time = 400

        # starting point
        self._current_x_pos = 40

        self._max_x_pos_memory = 0
        self._previous_x_pos_memory = 0
        self._steps_run_wrong_direction = 0

    def step(self, action):
        state, reward, done, info = self.env.step(action)
        score_diff = info["score"] - self._current_score
        reward += score_diff / 10.0

        time_diff = self._current_time - info["time"]

        if self._max_x_pos_memory < info["x_pos"]:
            self._max_x_pos_memory = info["x_pos"]
        else:
            self._steps_run_wrong_direction += time_diff

        #  Handle when the agent hits the left wall
        standstill = self._previous_x_pos_memory == info["x_pos"]
        if standstill:
            self._steps_run_wrong_direction += time_diff

        self._previous_x_pos_memory = info["x_pos"]

        # reward movement also for left direction /normal reward for left is -3
        # Make sure to force progress after moving in the wrong direction too long.
        if info["x_pos"] < self._current_x_pos:
            reward -= min((self._steps_run_wrong_direction / 10000), 1)
        elif standstill:
            reward -= 0.2
        """elif score_diff > 1:
            self._steps_run_wrong_direction = 0"""

        self._current_score = info["score"]
        self._current_x_pos = info["x_pos"]

        if done:
            if info["flag_get"]:
                reward += 350.0
            else:
                reward -= 100.0
                self._current_x_pos = 40
        return state, reward/10, done, info

In [ ]:
class CustomRewardNoMovingRightReward(gym.Wrapper):
    def __init__(self, env):
        super(CustomRewardNoMovingRightReward, self).__init__(env)
        self._current_score = 0
        self._number_of_lives = 2
        self._current_time = 400

        # starting point
        self._current_x_pos = 40
        self._max_x_pos_memory = 0
        self._previous_x_pos_memory = 0
        self._steps_run_wrong_direction = 0

    def step(self, action):
        def reset():
            self._current_x_pos = 40
            self._current_time = 400
            self._steps_run_wrong_direction = 0

        state, reward, done, info = self.env.step(action)
        score_diff = info["score"] - self._current_score
        life_loss = (self._number_of_lives - info["life"]) > 0

        # clip to avoid too high reward for mushroom/flower (1000p) and coins (200p)
        reward += min(score_diff, 150)

        time_diff = self._current_time - info["time"]

        if self._max_x_pos_memory < info["x_pos"]:
            self._max_x_pos_memory = info["x_pos"]
        else:
            self._steps_run_wrong_direction += time_diff

        #  Handle when the agent hits the left wall
        standstill = self._previous_x_pos_memory == info["x_pos"]
        if standstill:
            self._steps_run_wrong_direction += time_diff

        self._previous_x_pos_memory = info["x_pos"]

        # reward movement also for left direction /normal reward for left is -3
        # Make sure to force progress after moving in the wrong direction too long.
        if info["x_pos"] < self._current_x_pos:
            reward -= min((self._steps_run_wrong_direction / 100), 10)
        elif standstill:
            reward -= min((self._steps_run_wrong_direction / 100), 10)
        """elif score_diff > 1:
            self._steps_run_wrong_direction = 0"""

        self._current_score = info["score"]
        self._current_x_pos = info["x_pos"]
        self._current_time = info["time"]

        if done:
            if info["flag_get"]:
                reward += 350.0
            else:
                reward -= 100.0
                self._number_of_lives = 2
                reset()

        if life_loss:
            reward -= 50.0
            self._number_of_lives = info["life"]
            reset()

        return state, reward / 10, done, info

In [15]:
class CustomReward6(gym.Wrapper):
    def __init__(self, env):
        super(CustomReward6, self).__init__(env)
        self._current_score = 0
        self._number_of_lives = 2

    def step(self, action):
        state, reward, done, info = self.env.step(action)
        reward = (info["score"] - self._current_score) / 10

        if info["life"] == 255:
            life_loss = True
        else:
            life_loss = (self._number_of_lives - info["life"]) > 0

        self._current_score = info["score"]
        if done:
            if info["flag_get"]:
                reward += 350.0
            else:
                reward -= 100.0
                self._number_of_lives = 2
                self._current_score = 0

        if life_loss and not done:
            reward -= 50.0
            self._number_of_lives = info["life"]

        return state, reward, done, info

In [16]:
# Create the base environment
env = gym_super_mario_bros.make("SuperMarioBros-v0")
# My custom reward function
env = CustomReward6(env)
# Simplify the controls
customMovement = [['right', 'B'], ['right', 'A', 'B'], ['A'], ['left', 'B'], ['left', 'A', 'B']]
#env = JoypadSpace(env, customMovement)
env = JoypadSpace(env, SIMPLE_MOVEMENT)
# Grayscale
env = GrayScaleObservation(env, keep_dim=True)
# Wrap inside the Dummy Environment
env = DummyVecEnv([lambda: env])
# Stack the frames
env = VecFrameStack(env, 4, channels_order="last")

In [17]:
state = env.reset()

In [ ]:
SIMPLE_MOVEMENT[env.action_space.sample()]

In [ ]:
state, reward, done, info = env.step([env.action_space.sample()])
info

In [ ]:
"""plt.figure(figsize=(10, 8))
for idx in range(state.shape[3]):
    plt.subplot(1, 4, idx + 1)
    plt.imshow(state[0][:, :, idx])
plt.show()"""

In [7]:
# Import os for file path management
import os
# Import PPO for algos
from stable_baselines3 import PPO
# Import base callback for saving models
from stable_baselines3.common.callbacks import BaseCallback

In [8]:
class TrainAndLoggingCallback(BaseCallback):

    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)

        return True

In [9]:
CHECKPOINT_DIR = './train-reward-score-right-and-left13/'
LOG_DIR = './logs/train-reward-score-right-and-left13/'

In [10]:
# Setup model saving callback
callback = TrainAndLoggingCallback(check_freq=100000, save_path=CHECKPOINT_DIR)

In [11]:
# This is the AI model started
model = PPO('CnnPolicy', env, verbose=1, tensorboard_log=LOG_DIR, learning_rate=0.00001, n_steps=512) 

Using cuda device
Wrapping the env in a VecTransposeImage.


In [20]:
# Train the AI model, this is where the AI model starts to learn
model.learn(total_timesteps=10000000, callback=callback)

Logging to ./logs/PPO_10
----------------------------
| time/              |     |
|    fps             | 181 |
|    iterations      | 1   |
|    time_elapsed    | 2   |
|    total_timesteps | 512 |
----------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 114          |
|    iterations           | 2            |
|    time_elapsed         | 8            |
|    total_timesteps      | 1024         |
| train/                  |              |
|    approx_kl            | 0.0075984164 |
|    clip_fraction        | 0.0768       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.464       |
|    explained_variance   | 0.692        |
|    learning_rate        | 1e-06        |
|    loss                 | 110          |
|    n_updates            | 126960       |
|    policy_gradient_loss | 0.0152       |
|    value_loss           | 361          |
-----------------------------------------

------------------------------------------
| time/                   |              |
|    fps                  | 85           |
|    iterations           | 13           |
|    time_elapsed         | 78           |
|    total_timesteps      | 6656         |
| train/                  |              |
|    approx_kl            | 0.0044059525 |
|    clip_fraction        | 0.0375       |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.13        |
|    explained_variance   | 0.905        |
|    learning_rate        | 1e-06        |
|    loss                 | 31.6         |
|    n_updates            | 127070       |
|    policy_gradient_loss | 0.000397     |
|    value_loss           | 104          |
------------------------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 85           |
|    iterations           | 14           |
|    time_elapsed         | 84           |
|    total_

-----------------------------------------
| time/                   |             |
|    fps                  | 84          |
|    iterations           | 24          |
|    time_elapsed         | 145         |
|    total_timesteps      | 12288       |
| train/                  |             |
|    approx_kl            | 0.004654659 |
|    clip_fraction        | 0           |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.18       |
|    explained_variance   | -6.14       |
|    learning_rate        | 1e-06       |
|    loss                 | 0.414       |
|    n_updates            | 127180      |
|    policy_gradient_loss | -0.0016     |
|    value_loss           | 1.99        |
-----------------------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 84           |
|    iterations           | 25           |
|    time_elapsed         | 151          |
|    total_timesteps      | 1

-------------------------------------------
| time/                   |               |
|    fps                  | 83            |
|    iterations           | 35            |
|    time_elapsed         | 213           |
|    total_timesteps      | 17920         |
| train/                  |               |
|    approx_kl            | 0.00059351453 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -1.11         |
|    explained_variance   | 0.533         |
|    learning_rate        | 1e-06         |
|    loss                 | 17.1          |
|    n_updates            | 127290        |
|    policy_gradient_loss | -0.00135      |
|    value_loss           | 106           |
-------------------------------------------
-------------------------------------------
| time/                   |               |
|    fps                  | 83            |
|    iterations           | 36            |
|    time_elapsed         | 219 

------------------------------------------
| time/                   |              |
|    fps                  | 83           |
|    iterations           | 46           |
|    time_elapsed         | 281          |
|    total_timesteps      | 23552        |
| train/                  |              |
|    approx_kl            | 0.0013220884 |
|    clip_fraction        | 0            |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.22        |
|    explained_variance   | -1.72        |
|    learning_rate        | 1e-06        |
|    loss                 | 1.12         |
|    n_updates            | 127400       |
|    policy_gradient_loss | -0.000658    |
|    value_loss           | 2.94         |
------------------------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 83           |
|    iterations           | 47           |
|    time_elapsed         | 287          |
|    total_

...

-----------------------------------------
| time/                   |             |
|    fps                  | 83          |
|    iterations           | 695         |
|    time_elapsed         | 4265        |
|    total_timesteps      | 355840      |
| train/                  |             |
|    approx_kl            | 0.007143778 |
|    clip_fraction        | 0.00254     |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.04       |
|    explained_variance   | -4.3        |
|    learning_rate        | 1e-06       |
|    loss                 | 0.0877      |
|    n_updates            | 133890      |
|    policy_gradient_loss | -0.00588    |
|    value_loss           | 0.12        |
-----------------------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 83           |
|    iterations           | 696          |
|    time_elapsed         | 4271         |
|    total_timesteps      | 3

-------------------------------------------
| time/                   |               |
|    fps                  | 83            |
|    iterations           | 706           |
|    time_elapsed         | 4332          |
|    total_timesteps      | 361472        |
| train/                  |               |
|    approx_kl            | 0.00048929395 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -1.12         |
|    explained_variance   | -0.171        |
|    learning_rate        | 1e-06         |
|    loss                 | 0.0201        |
|    n_updates            | 134000        |
|    policy_gradient_loss | -0.000203     |
|    value_loss           | 0.0645        |
-------------------------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 83          |
|    iterations           | 707         |
|    time_elapsed         | 4338        

-------------------------------------------
| time/                   |               |
|    fps                  | 83            |
|    iterations           | 717           |
|    time_elapsed         | 4399          |
|    total_timesteps      | 367104        |
| train/                  |               |
|    approx_kl            | 0.00036329322 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -0.931        |
|    explained_variance   | -0.507        |
|    learning_rate        | 1e-06         |
|    loss                 | 0.342         |
|    n_updates            | 134110        |
|    policy_gradient_loss | -0.000502     |
|    value_loss           | 1.47          |
-------------------------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 83           |
|    iterations           | 718          |
|    time_elapsed         | 4405    

-----------------------------------------
| time/                   |             |
|    fps                  | 83          |
|    iterations           | 728         |
|    time_elapsed         | 4477        |
|    total_timesteps      | 372736      |
| train/                  |             |
|    approx_kl            | 0.004821861 |
|    clip_fraction        | 0           |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.07       |
|    explained_variance   | -0.418      |
|    learning_rate        | 1e-06       |
|    loss                 | 0.0285      |
|    n_updates            | 134220      |
|    policy_gradient_loss | -0.00387    |
|    value_loss           | 0.0995      |
-----------------------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 83          |
|    iterations           | 729         |
|    time_elapsed         | 4484        |
|    total_timesteps      | 373248

KeyboardInterrupt: 

In [ ]:
model.save('thisisatestmodel')

In [18]:
# Load model
model = PPO.load('./train/best_model_4500000')

In [19]:
model.set_env(env)

Wrapping the env in a VecTransposeImage.


In [ ]:
env = CustomReward6(env)

In [ ]:
state = env.reset()

In [ ]:
# Start the game 
state = env.reset()
# Loop through the game
while True:
    action, _ = model.predict(state)
    state, reward, done, info = env.step(action)
    env.render()
    if reward > 3 or reward < -1:
        print(reward)